In [1]:
#softmax: 在多分类中常用的激活函数，是基于逻辑回归的。
#softplus：softplus(x)=log(1+e^x)，近似生物神经激活函数，最近出现的。
#Relu：近似生物神经激活函数，最近出现的。
#tanh：双曲正切激活函数，也是很常用的。
#sigmoid：S型曲线激活函数，最常用的。
#hard_sigmoid：基于S型激活函数。
#linear：线性激活函数，最简单的

import pandas as pd
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.optimizers import SGD, Adam, RMSprop
from keras.utils import np_utils
import matplotlib.pyplot as plt

import matplotlib.pyplot as plt 
from matplotlib.backends.backend_pdf import PdfPages

class LossHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.losses = {'batch':[], 'epoch':[]}
        self.accuracy = {'batch':[], 'epoch':[]}
        self.val_loss = {'batch':[], 'epoch':[]}
        self.val_acc = {'batch':[], 'epoch':[]}

    def on_batch_end(self, batch, logs={}):
        self.losses['batch'].append(logs.get('loss'))
        self.accuracy['batch'].append(logs.get('acc'))
        self.val_loss['batch'].append(logs.get('val_loss'))
        self.val_acc['batch'].append(logs.get('val_acc'))

    def on_epoch_end(self, batch, logs={}):
        self.losses['epoch'].append(logs.get('loss'))
        self.accuracy['epoch'].append(logs.get('acc'))
        self.val_loss['epoch'].append(logs.get('val_loss'))
        self.val_acc['epoch'].append(logs.get('val_acc'))

    def loss_plot(self, loss_type):
        iters = range(len(self.losses[loss_type]))
        pdf = PdfPages('KNN.pdf')

        plt.figure()

        plt.rcParams['figure.dpi'] = 350

     # acc
        plt.plot(iters, self.accuracy[loss_type], 'r', label='train acc')
        # loss
        plt.plot(iters, self.losses[loss_type], 'g', label='train loss')
        if loss_type == 'epoch':
            # val_acc
            plt.plot(iters, self.val_acc[loss_type], 'b', label='val acc')
            # val_loss
            plt.plot(iters, self.val_loss[loss_type], 'k', label='val loss')
        plt.grid(True)
        plt.xlabel(loss_type)
        plt.ylabel('acc-loss')
        plt.legend(loc="center right")
        plt.title('Acc-loss curve of AD_KNN')
        pdf.savefig()
        plt.close()
        pdf.close()
        plt.show()
history = LossHistory()

Using TensorFlow backend.


In [14]:
import pandas as pd
import numpy as np

t_x=pd.read_csv('Result.csv', header=None, index_col=None)
t_y=pd.read_csv('Y_gai2.csv', header=None, index_col=None)

xx=t_x.values

xx_y=t_y.values


#xx = np.expand_dims(xx, axis=2)
#xx_y = np.expand_dims(xx_y, axis=2)

In [15]:
xx.shape

(17808, 27)

In [19]:
from keras.layers import Dense, Activation, Flatten, Convolution1D, Dropout
from sklearn.model_selection import train_test_split
import keras as K
from keras.layers import LSTM, Bidirectional

def creat_model():
    init = K.initializers.glorot_uniform()
    #scheduler = keras.callbacks.ReduceLROnPlateau(simple_adam, 'max', factor=0.5, patience=3)
    model = K.models.Sequential()


    #model.add(Convolution1D(nb_filter=64, filter_length=1, input_shape=(500, 10)))
    #model.add(Activation('relu'))
    #model.add(Flatten())
    #model.add(Dropout(0.4))
    # 定义卷积层

    # 编译模型
    model = keras.Sequential()



    model.add(K.layers.Dense(units=20 ,input_shape=(27,) ,kernel_initializer=init, activation='relu'))
    model.add(Dropout(0.2))
  #  model.add(K.layers.Dense(units=100 ,kernel_initializer=init, activation='relu'))
  #  model.add(Dropout(0.2))
    model.add(K.layers.Dense(units=1, kernel_initializer=init, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer=Adam(),metrics=['accuracy'])
    return model

from sklearn.model_selection import StratifiedKFold

def train():
    kfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=7)
    for train, test in kfold.split(xx, xx_y):
        train_x=xx[train]
        train_y=xx_y[train]
        test_x=xx[test]
        test_y=xx_y[test]
        train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, train_size=0.8, test_size=0.2, random_state=0)
        print(train_x.shape,test_x.shape,val_x.shape)
        model = creat_model()
        max_epochs = 10
        print("Starting training ")
        h = model.fit(train_x, train_y, epochs=max_epochs, batch_size=128,shuffle=True, verbose=1,callbacks=[history],validation_data=(val_x, val_y))
        print("Training finished \n")

        eval = model.evaluate(test_x, test_y, verbose=1)
        print("Evaluation on test data: loss = %0.6f accuracy = %0.2f%% \n" % (eval[0], eval[1] * 100) )
        print(model.summary())

def train2():
    train_x, test_x, train_y, test_y = train_test_split(xx, xx_y, train_size=0.7, test_size=0.3, random_state=0)
    train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, train_size=0.7, test_size=0.3, random_state=0)
    print(train_x.shape,test_x.shape,val_x.shape)
    model = creat_model()
    max_epochs = 5
    print("Starting training ")
    h = model.fit(train_x, train_y, epochs=max_epochs, batch_size=128,shuffle=True, verbose=1,callbacks=[history],validation_data=(val_x, val_y))
    print("Training finished \n")

    eval = model.evaluate(test_x, test_y, verbose=1)
    print("Evaluation on test data: loss = %0.6f accuracy = %0.2f%% \n" % (eval[0], eval[1] * 100) )
    print(model.summary())
 #   Z = model.predict(xx)
#    pd.DataFrame(Z).to_csv('Z.csv',header=None,index=False)
        
if __name__ == '__main__':
    train()
   # train2()



(9496, 27) (5937, 27) (2375, 27)
Starting training 
Train on 9496 samples, validate on 2375 samples
Epoch 1/10
9496/9496 [==============================] - 2s 207us/step - loss: 0.4097 - acc: 0.7931 - val_loss: 0.3558 - val_acc: 0.9124
Epoch 2/10
9496/9496 [==============================] - 0s 19us/step - loss: 0.2931 - acc: 0.9309 - val_loss: 0.2905 - val_acc: 0.9162
Epoch 3/10
9496/9496 [==============================] - 0s 21us/step - loss: 0.2494 - acc: 0.9318 - val_loss: 0.2727 - val_acc: 0.9187
Epoch 4/10
9496/9496 [==============================] - 0s 22us/step - loss: 0.2375 - acc: 0.9325 - val_loss: 0.2646 - val_acc: 0.9183
Epoch 5/10
9496/9496 [==============================] - 0s 19us/step - loss: 0.2299 - acc: 0.9319 - val_loss: 0.2620 - val_acc: 0.9175
Epoch 6/10
9496/9496 [==============================] - 0s 19us/step - loss: 0.2231 - acc: 0.9322 - val_loss: 0.2643 - val_acc: 0.9149
Epoch 7/10
9496/9496 [==============================] - 0s 19us/step - loss: 0.2209 - acc

In [11]:
history.loss_plot('epoch')


In [4]:

import matplotlib.pyplot as plt 
from matplotlib.backends.backend_pdf import PdfPages
 
pdf = PdfPages('zz.pdf')
 
plt.figure()

plt.rcParams['figure.dpi'] = 350


plt.plot([0,1],[0,1],'k--')


 
pdf.savefig()
plt.close()
pdf.close()
plt.show()

In [7]:
from sklearn import svm
clf = svm.SVC(kernel='rbf', C=1).fit(train_x, train_y)

print ("svm :",clf.score(test_x, test_y))

ValueError: Found array with dim 3. Estimator expected <= 2.

In [6]:
        train_x, test_x, train_y, test_y = train_test_split(xx, xx_y, train_size=0.7, test_size=0.3, random_state=0)


In [13]:
Z = model.predict(xx)
pd.DataFrame(Z).to_csv('Z.csv',header=None,index=False)

NameError: name 'model' is not defined